In [1]:
# !spacy download en_core_web_sm
# !spacy download en_core_web_md

In [39]:
import json
from collections import Counter
from pathlib import Path
from pydantic import BaseModel
from typing import Union, TypedDict, Dict
from textwrap import dedent

from ollama import Client
from langchain_ollama import ChatOllama
import spacy

In [3]:
# MODEL_NAME = "gemma3:27b"
MODEL_NAME = "command-a"

llm = ChatOllama(
    base_url="http://host.docker.internal:11434",
    model=MODEL_NAME,
    num_ctx=10000,
    # format="json"
)

json_llm = ChatOllama(
    base_url="http://host.docker.internal:11434",
    model=MODEL_NAME,
    num_ctx=10000,
    format="json"
)

In [4]:
# Load the English SpaCy model
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")


/workspace/.venv/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
# Sample excerpt for demonstration purposes. Replace with a more detailed summary for a richer graph.
 
text = """\
The key issues and events we examined include the following:
The Campaign’s response to reports about Russian support for Trump. During the 2016
presidential campaign, questions arose about the Russian government’s apparent support for
candidate Trump. After WikiLeaks released politically damaging Democratic Party emails that
were reported to have been hacked by Russia, Trump publicly expressed skepticism that Russia
was responsible for the hacks at the same time that he and other Campaign officials privately
sought information from Trump associate Roger Stone about any further planned WikiLeaks
releases. Trump also denied having any business in or connections to Russia, even though as late
as June 2016 the Trump Organization had been pursuing a licensing deal for a skyscraper to be
built in Russia called Trump Tower Moscow. After the election, the President expressed concerns
to advisors that reports of Russia’s election interference might lead the public to question the
legitimacy of his election.
Conduct involving FBI Director Comey and Michael Flynn. In mid-January 2017,
incoming National Security Advisor Michael Flynn falsely denied to the Vice President, other
administration officials, and FBI agents that he had talked to Russian Ambassador Sergey Kislyak
about Russia’s response to U.S. sanctions on Russia for its election interference. On January 27,
the day after the President was told that Flynn had lied to the Vice President and had made similar
statements to the FBI, the President invited FBI Director Comey to a private dinner at the White
House and told Comey that he needed loyalty. On February 14, the day after the President
requested Flynn’s resignation, the President told an outside advisor, “Now that we fired Flynn, the
Russia thing is over.” The advisor disagreed and said the investigations would continue.
Later that afternoon, the President cleared the Oval Office to have a one-on-one meeting
with Comey. Referring to the FBI’s investigation of Flynn, the President said, “I hope you can
see your way clear to letting this go, to letting Flynn go. He is a good guy. I hope you can let this
go.” Shortly after requesting Flynn’s resignation and speaking privately to Comey, the President
sought to have Deputy National Security Advisor K.T. McFarland draft an internal letter stating
that the President had not directed Flynn to discuss sanctions with Kislyak. McFarland declined
because she did not know whether that was true, and a White House Counsel’s Office attorney
thought that the request would look like a quid pro quo for an ambassadorship she had been offered.
The President’s reaction to the continuing Russia investigation. In February 2017,
Attorney General Jeff Sessions began to assess whether he had to recuse himself from campaign-
related investigations because of his role in the Trump Campaign. In early March, the President
told White House Counsel Donald McGahn to stop Sessions from recusing. And after Sessions
announced his recusal on March 2, the President expressed anger at the decision and told advisors
that he should have an Attorney General who would protect him. That weekend, the President
took Sessions aside at an event and urged him to “unrecuse.” Later in March, Comey publicly
3
U.S. Department of Justice
Attorney Work Product // May Contain Material Protected Under Fed. R. Crim. P. 6(e)______________________________________________________
disclosed at a congressional hearing that the FBI was investigating “the Russian government’s
efforts to interfere in the 2016 presidential election,” including any links or coordination between
the Russian government and the Trump Campaign. In the following days, the President reached
out to the Director of National Intelligence and the leaders of the Central Intelligence Agency
(CIA) and the National Security Agency (NSA) to ask them what they could do to publicly dispel
the suggestion that the President had any connection to the Russian election-interference effort.
The President also twice called Comey directly, notwithstanding guidance from McGahn to avoid
direct contacts with the Department of Justice. Comey had previously assured the President that
the FBI was not investigating him personally, and the President asked Comey to “lift the cloud”
of the Russia investigation by saying that publicly.
The President’s termination of Comey. On May 3, 2017, Comey testified in a
congressional hearing, but declined to answer questions about whether the President was
personally under investigation. Within days, the President decided to terminate Comey. The
President insisted that the termination letter, which was written for public release, state that Comey
had informed the President that he was not under investigation. The day of the firing, the White
House maintained that Comey’s termination resulted from independent recommendations from the
Attorney General and Deputy Attorney General that Comey should be discharged for mishandling
the Hillary Clinton email investigation. But the President had decided to fire Comey before
hearing from the Department of Justice. The day after firing Comey, the President told Russian
officials that he had “faced great pressure because of Russia,” which had been “taken off” by
Comey’s firing. The next day, the President acknowledged in a television interview that he was
going to fire Comey regardless of the Department of Justice’s recommendation and that when he
“decided to just do it,” he was thinking that “this thing with Trump and Russia is a made-up story.”
In response to a question about whether he was angry with Comey about the Russia investigation,
the President said, “As far as I’m concerned, I want that thing to be absolutely done properly,”
adding that firing Comey “might even lengthen out the investigation.”
The appointment of a Special Counsel and efforts to remove him. On May 17, 2017, the
Acting Attorney General for the Russia investigation appointed a Special Counsel to conduct the
investigation and related matters. The President reacted to news that a Special Counsel had been
appointed by telling advisors that it was “the end of his presidency” and demanding that Sessions
resign. Sessions submitted his resignation, but the President ultimately did not accept it. The
President told aides that the Special Counsel had conflicts of interest and suggested that the Special
Counsel therefore could not serve. The President’s advisors told him the asserted conflicts were
meritless and had already been considered by the Department of Justice.
On June 14, 2017, the media reported that the Special Counsel’s Office was investigating
whether the President had obstructed justice. Press reports called this “a major turning point” in
the investigation: while Comey had told the President he was not under investigation, following
Comey’s firing, the President now was under investigation. The President reacted to this news
with a series of tweets criticizing the Department of Justice and the Special Counsel’s
investigation. On June 17, 2017, the President called McGahn at home and directed him to call
the Acting Attorney General and say that the Special Counsel had conflicts of interest and must be
removed. McGahn did not carry out the direction, however, deciding that he would resign rather
than trigger what he regarded as a potential Saturday Night Massacre.
4
U.S. Department of Justice
Attorney Work Product // May Contain Material Protected Under Fed. R. Crim. P. 6(e)______________________________________________________
Efforts to curtail the Special Counsel’s investigation. Two days after directing McGahn
to have the Special Counsel removed, the President made another attempt to affect the course of
the Russia investigation. On June 19, 2017, the President met one-on-one in the Oval Office with
his former campaign manager Corey Lewandowski, a trusted advisor outside the government, and
dictated a message for Lewandowski to deliver to Sessions. The message said that Sessions should
publicly announce that, notwithstanding his recusal from the Russia investigation, the investigation
was “very unfair” to the President, the President had done nothing wrong, and Sessions planned to
meet with the Special Counsel and “let [him] move forward with investigating election meddling
for future elections.” Lewandowski said he understood what the President wanted Sessions to do.
One month later, in another private meeting with Lewandowski on July 19, 2017, the
President asked about the status of his message for Sessions to limit the Special Counsel
investigation to future election interference. Lewandowski told the President that the message
would be delivered soon. Hours after that meeting, the President publicly criticized Sessions in an
interview with the New York Times, and then issued a series of tweets making it clear that
Sessions’s job was in jeopardy. Lewandowski did not want to deliver the President’s message
personally, so he asked senior White House official Rick Dearborn to deliver it to Sessions.
Dearborn was uncomfortable with the task and did not follow through.
Efforts to prevent public disclosure of evidence. In the summer of 2017, the President
learned that media outlets were asking questions about the June 9, 2016 meeting at Trump Tower
between senior campaign officials, including Donald Trump Jr., and a Russian lawyer who was
said to be offering damaging information about Hillary Clinton as “part of Russia and its
government’s support for Mr. Trump.” On several occasions, the President directed aides not to
publicly disclose the emails setting up the June 9 meeting, suggesting that the emails would not
leak and that the number of lawyers with access to them should be limited. Before the emails
became public, the President edited a press statement for Trump Jr. by deleting a line that
acknowledged that the meeting was with “an individual who [Trump Jr.] was told might have
information helpful to the campaign” and instead said only that the meeting was about adoptions
of Russian children. When the press asked questions about the President’s involvement in Trump
Jr.’s statement, the President’s personal lawyer repeatedly denied the President had played any
role.
Further efforts to have the Attorney General take control of the investigation. In early
summer 2017, the President called Sessions at home and again asked him to reverse his recusal
from the Russia investigation. Sessions did not reverse his recusal. In October 2017, the President
met privately with Sessions in the Oval Office and asked him to “take [a] look” at investigating
Clinton. In December 2017, shortly after Flynn pleaded guilty pursuant to a cooperation
agreement, the President met with Sessions in the Oval Office and suggested, according to notes
taken by a senior advisor, that if Sessions unrecused and took back supervision of the Russia
investigation, he would be a “hero.” The President told Sessions, “I’m not going to do anything
or direct you to do anything. I just want to be treated fairly.” In response, Sessions volunteered
that he had never seen anything “improper” on the campaign and told the President there was a
“whole new leadership team” in place. He did not unrecuse.
Efforts to have McGahn deny that the President had ordered him to have the Special
Counsel removed. In early 2018, the press reported that the President had directed McGahn to
5
U.S. Department of Justice
Attorney Work Product // May Contain Material Protected Under Fed. R. Crim. P. 6(e)
have the Special Counsel removed in June 2017 and that McGahn had threatened to resign rather
than carry out the order. The President reacted to the news stories by directing White House
officials to tell McGahn to dispute the story and create a record stating he had not been ordered to
have the Special Counsel removed. McGahn told those officials that the media reports were
accurate in stating that the President had directed McGahn to have the Special Counsel removed.
The President then met with McGahn in the Oval Office and again pressured him to deny the
reports. In the same meeting, the President also asked McGahn why he had told the Special
Counsel about the President’s effort to remove the Special Counsel and why McGahn took notes
of his conversations with the President. McGahn refused to back away from what he remembered
happening and perceived the President to be testing his mettle.
Conduct towards Flynn, Manafort, and Stone. After Flynn withdrew from a joint defense
agreement with the President and began cooperating with the government, the President’s personal
counsel left a message for Flynn’s attorneys reminding them of the President’s warm feelings
towards Flynn, which he said “still remains,” and asking for a “heads up” if Flynn knew
“information that implicates the President.” When Flynn’s counsel reiterated that Flynn could no
longer share information pursuant to a joint defense agreement, the President’s personal counsel
said he would make sure that the President knew that Flynn’s actions reflected “hostility” towards
the President. During Manafort’s prosecution and when the jury in his criminal trial was
deliberating, the President praised Manafort in public, said that Manafort was being treated
unfairly, and declined to rule out a pardon. After Manafort was convicted, the President called
Manafort “a brave man” for refusing to “break” and said that “flipping” “almost ought to be
outlawed.” After Roger Stone publicly announced that he would not cooperate with prosecutors,
the President called Stone “very brave” and said he had “guts!” for not “testify[ing] against
Trump.”
Conduct involving Michael Cohen. The President’s conduct towards Michael Cohen, a
former Trump Organization executive, changed from praise for Cohen when he falsely minimized
the President’s involvement in the Trump Tower Moscow project, to castigation of Cohen when
he became a cooperating witness. From September 2015 to June 2016, Cohen had pursued the
Trump Tower Moscow project on behalf of the Trump Organization and had briefed candidate
Trump on the project numerous times, including discussing whether Trump should travel to Russia
to advance the deal. In 2017, Cohen provided false testimony to Congress about the project,
including stating that he had only briefed Trump on the project three times and never discussed
travel to Russia with him, in an effort to adhere to a “party line” that Cohen said was developed to
minimize the President’s connections to Russia. While preparing for his congressional testimony,
Cohen had extensive discussions with the President’s personal counsel, who, according to Cohen,
said that Cohen should “stay on message” and not contradict the President. After the FBI searched
Cohen’s home and office in April 2018, the President publicly asserted that Cohen would not
“flip,” contacted him directly to tell him to “stay strong,” and privately passed messages of support
to him. Cohen also discussed pardons with the President’s personal counsel and believed that if
he stayed on message he would be taken care of. But after Cohen began cooperating with the
government in the summer of 2018, the President publicly criticized him, called him a “rat,” and
suggested that his family members had committed crimes.
6
U.S. Department of Justice
Attorney Work Product // May Contain Material Protected Under Fed. R. Crim. P. 6(e)______________________________________________________
Overarching factual issues. We did not make a traditional prosecution decision about
these facts, but the evidence we obtained supports several general statements about the President’s
conduct.
Several features of the conduct we investigated distinguish it from typical obstruction-of-
justice cases. First, the investigation concerned the President, and some of his actions, such as
firing the FBI director, involved facially lawful acts within his Article II authority, which raises
constitutional issues discussed below. At the same time, the President’s position as the head of
the Executive Branch provided him with unique and powerful means of influencing official
proceedings, subordinate officers, and potential witnesses—all of which is relevant to a potential
obstruction-of-justice analysis. Second, unlike cases in which a subject engages in obstruction of
justice to cover up a crime, the evidence we obtained did not establish that the President was
involved in an underlying crime related to Russian election interference. Although the obstruction
statutes do not require proof of such a crime, the absence of that evidence affects the analysis of
the President’s intent and requires consideration of other possible motives for his conduct. Third,
many of the President’s acts directed at witnesses, including discouragement of cooperation with
the government and suggestions of possible future pardons, took place in public view. That
circumstance is unusual, but no principle of law excludes public acts from the reach of the
obstruction laws. If the likely effect of public acts is to influence witnesses or alter their testimony,
the harm to the justice system’s integrity is the same.
Although the series of events we investigated involved discrete acts, the overall pattern of
the President’s conduct towards the investigations can shed light on the nature of the President’s
acts and the inferences that can be drawn about his intent. In particular, the actions we investigated
can be divided into two phases, reflecting a possible shift in the President’s motives. The first
phase covered the period from the President’s first interactions with Comey through the President’s
firing of Comey. During that time, the President had been repeatedly told he was not personally
under investigation. Soon after the firing of Comey and the appointment of the Special Counsel,
however, the President became aware that his own conduct was being investigated in an
obstruction-of-justice inquiry. At that point, the President engaged in a second phase of conduct,
involving public attacks on the investigation, non-public efforts to control it, and efforts in both
public and private to encourage witnesses not to cooperate with the investigation. Judgments about
the nature of the President’s motives during each phase would be informed by the totality of the
evidence.
"""



In [6]:
summary = llm.invoke(f"""\
Text:
{text}

Summarize the text above, focusing on people, organizations, relationships (peolpe<->people, people<->organizations, organizations<->organizations), and the actions of people and organizations. 
Respond in regular paragraphs using complete, detailed sentences.""")

In [7]:
print(summary.content)

The text details a series of events involving President Donald Trump, his campaign associates, administration officials, and law enforcement agencies, primarily centered around the investigation into Russian interference in the 2016 U.S. presidential election. Key individuals include Trump, FBI Director James Comey, National Security Advisor Michael Flynn, Attorney General Jeff Sessions, White House Counsel Don McGahn, Special Counsel Robert Mueller, and several others like Roger Stone, Paul Manafort, Michael Cohen, and Corey Lewandowski.

During the 2016 campaign, Trump publicly downplayed Russian involvement in hacking Democratic Party emails while privately seeking information about further WikiLeaks releases through associate Roger Stone. Despite denying business ties to Russia, Trump’s organization pursued a Moscow tower project until mid-2016. After the election, Trump expressed concern that reports of Russian interference might undermine his legitimacy as president.

In January 

In [8]:
# Process the summary text
doc = nlp(summary.content)

In [9]:
[ent.text for ent in doc.ents]

['Donald Trump',
 'Russian',
 '2016',
 'U.S.',
 'Trump',
 'FBI',
 'James Comey',
 'National Security',
 'Michael Flynn',
 'Jeff Sessions',
 'White House',
 'Don McGahn',
 'Robert Mueller',
 'Roger Stone',
 'Paul Manafort',
 'Michael Cohen',
 'Corey Lewandowski',
 '2016',
 'Trump',
 'Russian',
 'Democratic Party',
 'WikiLeaks',
 'Roger Stone',
 'Russia',
 'Trump',
 'Moscow',
 'mid-2016',
 'Trump',
 'Russian',
 'January 2017',
 'Flynn',
 'Russian',
 'Sergey Kislyak',
 'Trump',
 'FBI',
 'Comey',
 'Flynn',
 'one',
 'Trump',
 'Comey',
 'May 2017',
 'Russia',
 'the Department of Justice',
 'Trump',
 'Comey',
 'Russia',
 'Trump',
 'Russia',
 'McGahn',
 'Mueller',
 'McGahn',
 'Lewandowski',
 'Trump',
 '2016',
 'Trump Tower',
 'Russian',
 'Hillary Clinton',
 'Trump',
 'Manafort',
 'Stone',
 'Cohen',
 'Moscow',
 'Cohen',
 'Trump',
 'two',
 'Trump',
 'Comey',
 'second',
 'Russian',
 'Trump']

In [10]:
# split document into sentences
def split_document_sent(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

# define custom relationship extraction and text processing
def process_text(text, verbose=False):
    doc = nlp(text)
    if verbose:
        print(f"Text: {doc.text}")
        print(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")
        # Relations extraction logic can be added here
    return doc

def create_deduplication_mapper(entities:list[str]):
    
    prompt = dedent(f"""\
    Entity Names:
    {entities}
    
    Above is a list of entity names. Some of the entites are implicitly duplicates \
    Create and return mapper that maps existing, non-standardized or shortened versions of people's or organization's \
    names to a new full and proper standardized entity name. \
    The mapper should be formatted as a dict[str, str] where the key is the existing name as it appears in the list above and the value is the standardized, full proper name.\ 
    Return the mapper only.
    """)
                    
    response = json_llm.invoke(prompt)
    output = json.loads(response.content)
    return output
    

# Pipeline to run entity extraction
def extract_entities(text, verbose=False):
    processed_data = []
    entity_counts = Counter()

    sentences = split_document_sent(text)
    unique_entities = set()
    for sent in sentences:
        doc = process_text(sent, verbose)
        entities = [(ent.text, ent.label_) for ent in doc.ents]

        # Store processed data for each sentence
        processed_data.append({'text': doc.text, 'entities': entities})
        unique_entities = unique_entities | set([e[0] for e in entities])

        # Update counters
        entity_counts.update([ent[1] for ent in entities])

    # Deduplicate entites in processed data
    deduplicated_entity_mapper = create_deduplication_mapper(unique_entities)
    for sentence in processed_data:
        entities = sentence["entities"]
        for i in range(len(entities)):
            ent_name = entities[i][0]
            if ent_name in deduplicated_entity_mapper:
                new_name = deduplicated_entity_mapper[ent_name]
                print(f"Mapping '{ent_name}' => '{new_name}'")
                entities[i] = (new_name, entities[i][1])
    
    print(processed_data)
    
    # Export to JSON
    with open('processed_data.json', 'w') as f:
        json.dump(processed_data, f)

    # Display summary
    print(f"Entity counts: {entity_counts}")

# Run the pipeline on the summary text
verbose = False
extract_entities(summary.content, verbose)

Mapping 'Donald Trump' => 'Donald Trump'
Mapping 'Russian' => 'Russia'
Mapping '2016' => '2016'
Mapping 'U.S.' => 'United States'
Mapping 'Trump' => 'Donald Trump'
Mapping 'FBI' => 'Federal Bureau of Investigation (FBI)'
Mapping 'James Comey' => 'James Comey'
Mapping 'National Security' => 'National Security'
Mapping 'Michael Flynn' => 'Michael Flynn'
Mapping 'Jeff Sessions' => 'Jeff Sessions'
Mapping 'White House' => 'White House'
Mapping 'Don McGahn' => 'Don McGahn'
Mapping 'Robert Mueller' => 'Robert Mueller'
Mapping 'Roger Stone' => 'Roger Stone'
Mapping 'Paul Manafort' => 'Paul Manafort'
Mapping 'Corey Lewandowski' => 'Corey Lewandowski'
Mapping '2016' => '2016'
Mapping 'Trump' => 'Donald Trump'
Mapping 'Russian' => 'Russia'
Mapping 'Democratic Party' => 'Democratic Party'
Mapping 'WikiLeaks' => 'WikiLeaks'
Mapping 'Roger Stone' => 'Roger Stone'
Mapping 'Russia' => 'Russia'
Mapping 'Trump' => 'Donald Trump'
Mapping 'Moscow' => 'Moscow'
Mapping 'mid-2016' => 'Mid-2016'
Mapping 'Tru

In [49]:
## Create node and relationships using LLMs

# Load processed data from JSON
json_path = Path("processed_data.json")
with open(json_path, "r") as f:
    processed_data = json.load(f)

# Prepare nodes and relationships
nodes = []
relationships = []

# Formulate a prompt
prompt = (
    "JSON data:\n"
    f"```json\n{json.dumps(processed_data)}```\n\n"
    
    "Extract entities and relationships from the JSON data above. For each entry in data['entities'], "
    "create a 'node' dictionary with fields 'id' (unique identifier), 'name' (entity text), and 'type' (entity label). "
    "For entities that have meaningful connections, define 'relationships' as dictionaries with 'source' (source node id), "
    "'target' (target node id), 'relationship' (type of connection), and 'text' (exact copy of the existing text field). Create max 30 nodes, format 'relationships' in the format of capital letters and _ inbetween words and format the entire response in the JSON output containing only variables nodes and relationships without any text inbetween"
)



In [20]:
print(prompt)

JSON data:
```json
[{"text": "The text details a series of events involving President Donald Trump, his campaign associates, administration officials, and law enforcement agencies, primarily centered around the investigation into Russian interference in the 2016 U.S. presidential election.", "entities": [["Donald Trump", "PERSON"], ["Russia", "NORP"], ["2016", "DATE"], ["United States", "GPE"]]}, {"text": "Key individuals include Trump, FBI Director James Comey, National Security Advisor Michael Flynn, Attorney General Jeff Sessions, White House Counsel Don McGahn, Special Counsel Robert Mueller, and several others like Roger Stone, Paul Manafort, Michael Cohen, and Corey Lewandowski.", "entities": [["Donald Trump", "ORG"], ["Federal Bureau of Investigation (FBI)", "ORG"], ["James Comey", "PERSON"], ["National Security", "ORG"], ["Michael Flynn", "PERSON"], ["Jeff Sessions", "PERSON"], ["White House", "ORG"], ["Don McGahn", "PERSON"], ["Robert Mueller", "PERSON"], ["Roger Stone", "PERS

In [21]:
# class Node(BaseModel):
#     type: str
#     id: Union[str, int]
#     name: str

# class Relationship(BaseModel):
#     source: Union[str, int]
#     target: Union[str, int]
#     relationship: str

# class GraphModel(BaseModel):
#     nodes: list[Node] = []
#     relationships: list[Relationship] = []
# output_constrained_llm = llm.with_structured_output(GraphModel)

In [22]:
response = json_llm.invoke(
   [
        {"role": "system", "content": "You are a helpful assistant that structures data into nodes and relationships."},
        {"role": "user", "content": prompt}
    ]
)
text = response.content #.replace("```json", "").replace("```","")
output = json.loads(text)


In [50]:
# Populate nodes and relationships lists
nodes.extend(output.get("nodes", []))
relationships.extend(output.get("relationships", []))

In [51]:
relationships[0]

{'source': 1,
 'target': 2,
 'relationship': 'INVOLVED_WITH',
 'text': 'The text details a series of events involving President Donald Trump, his campaign associates, administration officials, and law enforcement agencies, primarily centered around the investigation into Russian interference in the 2016 U.S. presidential election.'}

In [53]:
relationships[0]

{'source': 1,
 'target': 2,
 'relationship': 'INVOLVED_WITH',
 'text': 'The text details a series of events involving President Donald Trump, his campaign associates, administration officials, and law enforcement agencies, primarily centered around the investigation into Russian interference in the 2016 U.S. presidential election.',
 'embedding': [-0.0083912825,
  0.040449597,
  -0.16651492,
  0.016698223,
  0.09478998,
  -0.0064174263,
  -0.035186846,
  -0.00026054532,
  0.060705286,
  -0.04951203,
  -0.05450845,
  0.01361221,
  0.05700782,
  0.02808935,
  -0.0050570057,
  -0.02371109,
  0.002384475,
  -0.02065644,
  -0.08198298,
  0.031601727,
  0.0216582,
  -0.0058269836,
  -0.055907577,
  0.0135463495,
  0.083864875,
  0.041668978,
  0.01190636,
  -0.04226339,
  -0.036841076,
  -0.039984636,
  -0.008000949,
  0.00015211376,
  -0.043493386,
  -0.02387685,
  -0.06780507,
  -0.06955301,
  0.054376684,
  0.0143086305,
  -0.0028356519,
  -0.012753904,
  0.040278327,
  0.04156093,
  -0.0

In [52]:
 # Define a function to compute embeddings
client = Client(host="http://host.docker.internal:11434")
def compute_embeddings(texts:list[str]):
    return client.embed(model="nomic-embed-text", input=texts).model_dump()["embeddings"]
 
# Generate embeddings for Nolan movies
texts = [r["text"] for r in relationships]
embeddings = compute_embeddings(texts)
for i, emb in enumerate(embeddings):
    relationships[i]["embedding"] = emb

# Generate Queries

In [56]:
def generate_cypher_queries(nodes, relationships):
    queries = []

    # Create nodes
    for node in nodes:
        query = f"CREATE (n:{node['type']} {{id: '{node['id']}', name: '{node['name']}'}})"
        queries.append(query)

    # Create relationships
    for rel in relationships:
        query = f"MATCH (a {{id: '{rel['source']}'}}), (b {{id: '{rel['target']}'}}) " \
                f"CREATE (a)-[r:{rel['relationship']}]->(b)" \
                f'SET r = {{text: \'{rel["text"]}\', embedding: {rel["embedding"]}}}'
        queries.append(query)

    return queries

cypher_queries = generate_cypher_queries(nodes, relationships)
print(cypher_queries)

["CREATE (n:PERSON {id: '1', name: 'Donald Trump'})", "CREATE (n:NORP {id: '2', name: 'Russia'})", "CREATE (n:DATE {id: '3', name: '2016'})", "CREATE (n:GPE {id: '4', name: 'United States'})", "CREATE (n:ORG {id: '5', name: 'Federal Bureau of Investigation (FBI)'})", "CREATE (n:PERSON {id: '6', name: 'James Comey'})", "CREATE (n:ORG {id: '7', name: 'National Security'})", "CREATE (n:PERSON {id: '8', name: 'Michael Flynn'})", "CREATE (n:PERSON {id: '9', name: 'Jeff Sessions'})", "CREATE (n:ORG {id: '10', name: 'White House'})", "CREATE (n:PERSON {id: '11', name: 'Don McGahn'})", "CREATE (n:PERSON {id: '12', name: 'Robert Mueller'})", "CREATE (n:PERSON {id: '13', name: 'Roger Stone'})", "CREATE (n:PERSON {id: '14', name: 'Paul Manafort'})", "CREATE (n:PERSON {id: '15', name: 'Michael Cohen'})", "CREATE (n:PERSON {id: '16', name: 'Corey Lewandowski'})", "CREATE (n:ORG {id: '17', name: 'Democratic Party'})", "CREATE (n:ORG {id: '18', name: 'WikiLeaks'})", "CREATE (n:GPE {id: '19', name: 'M

# Execute Queries

In [57]:
from neo4j import GraphDatabase

# Initialize the Neo4j driver for Memgraph (modify the URI if necessary)
uri = "bolt://host.docker.internal:7687"
user = ""
password = ""
driver = GraphDatabase.driver(uri, auth=(user, password))

# Function to execute Cypher queries in Memgraph
def execute_cypher_queries(queries):
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n;")
        for query in queries:
            try:
                session.run(query)
                print(f"Executed query: {query}")
            except Exception as e:
                print(f"Error executing query: {query}. Error: {e}")

# Execute the generated Cypher queries
execute_cypher_queries(cypher_queries)

Executed query: CREATE (n:PERSON {id: '1', name: 'Donald Trump'})
Executed query: CREATE (n:NORP {id: '2', name: 'Russia'})
Executed query: CREATE (n:DATE {id: '3', name: '2016'})
Executed query: CREATE (n:GPE {id: '4', name: 'United States'})
Executed query: CREATE (n:ORG {id: '5', name: 'Federal Bureau of Investigation (FBI)'})
Executed query: CREATE (n:PERSON {id: '6', name: 'James Comey'})
Executed query: CREATE (n:ORG {id: '7', name: 'National Security'})
Executed query: CREATE (n:PERSON {id: '8', name: 'Michael Flynn'})
Executed query: CREATE (n:PERSON {id: '9', name: 'Jeff Sessions'})
Executed query: CREATE (n:ORG {id: '10', name: 'White House'})
Executed query: CREATE (n:PERSON {id: '11', name: 'Don McGahn'})
Executed query: CREATE (n:PERSON {id: '12', name: 'Robert Mueller'})
Executed query: CREATE (n:PERSON {id: '13', name: 'Roger Stone'})
Executed query: CREATE (n:PERSON {id: '14', name: 'Paul Manafort'})
Executed query: CREATE (n:PERSON {id: '15', name: 'Michael Cohen'})
Ex